In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import os

In [2]:
class CelebA_Attr(torch.utils.data.Dataset):
    def __init__(self, root_dir, train_or_test, restricted_degree=0):
        self._root_dir = root_dir
        self._train_or_test = train_or_test
        
        info_pak = np.load(os.path.join(self._root_dir, 'celeba_attr.npz'))
        train_idxs = info_pak['train_idxs']
        val_idxs = info_pak['val_idxs']
        test_idxs = info_pak['test_idxs']
        
        attribute_names = info_pak['attribute_names']
        attributes = info_pak['attributes']
        male_attr_idx = 20
        
        def get_label(idxs, restricted_degree):
            def jj(attr):
                important_attributes_idx = [0, 1, 4, 9, 16, 18, 22, 24, 29, 30, 34, 36, 37, 38]
                x = np.array([0 for i in range(attr.shape[0])])
                for i in important_attributes_idx:
                    x = x + attr[:, i]
                return x

            label = attributes[idxs]
            sig = jj(label) >= restricted_degree
            label = label[sig]

            data = np.delete(label, [male_attr_idx], 1)
            label = label[:, male_attr_idx]
            return data.astype('float32'), label
        
        if self._train_or_test=='train':
            self._data, self._label = get_label(train_idxs, restricted_degree)
        elif self._train_or_test=='test':
            self._data, self._label = get_label(test_idxs, restricted_degree)
    
    def __len__(self):
        return self._label.shape[0]
        
    def __getitem__(self, index):
        return self._data[index], self._label[index]
        

In [3]:
# Training settings
batch_size = 32
test_batch_size = 1000
epochs = 10
lr = 0.001
eps = 1e-8

save_model = False

In [4]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    CelebA_Attr('./data/toy_celeba', 'train'),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    CelebA_Attr('./data/toy_celeba', 'test'),
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = torch.nn.Sequential(
          torch.nn.Linear(39, 200),
          torch.nn.ReLU(),
          torch.nn.Linear(200, 2),
          torch.nn.LogSoftmax(dim=1)
        ).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, eps=eps)

In [5]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    running_loss = 0.0
    running_corrects = 0
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output, target)
        _, preds = torch.max(output, 1)
#         _, labels = torch.max(target, 1)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    
    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'train', loss, epoch_acc))

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
criterion = nn.CrossEntropyLoss(reduction='sum')

for epoch in range(1, epochs + 1):
    print('Epoch {}'.format(epoch))
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

Epoch 1
train Loss: 1.2291 Acc: 0.9316
Test set: Average loss: 0.1330, Accuracy: 18731/19962 (93.83%)

Epoch 2
train Loss: 2.9307 Acc: 0.9355
Test set: Average loss: 0.1379, Accuracy: 18691/19962 (93.63%)

Epoch 3
train Loss: 3.4165 Acc: 0.9360
Test set: Average loss: 0.1330, Accuracy: 18755/19962 (93.95%)

Epoch 4
train Loss: 3.2261 Acc: 0.9376
Test set: Average loss: 0.1339, Accuracy: 18737/19962 (93.86%)

Epoch 5
train Loss: 0.8464 Acc: 0.9376
Test set: Average loss: 0.1313, Accuracy: 18780/19962 (94.08%)

Epoch 6
train Loss: 3.2316 Acc: 0.9380
Test set: Average loss: 0.1345, Accuracy: 18755/19962 (93.95%)

Epoch 7
train Loss: 1.7594 Acc: 0.9383
Test set: Average loss: 0.1338, Accuracy: 18760/19962 (93.98%)

Epoch 8
train Loss: 0.7600 Acc: 0.9384
Test set: Average loss: 0.1329, Accuracy: 18767/19962 (94.01%)

Epoch 9
train Loss: 7.2091 Acc: 0.9392
Test set: Average loss: 0.1353, Accuracy: 18735/19962 (93.85%)

Epoch 10
train Loss: 1.0288 Acc: 0.9397
Test set: Average loss: 0.1343, A

In [7]:
if (save_model):
    torch.save(model.state_dict(),"mnist_cnn.pt")